In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/clean_raw_data.csv
/kaggle/input/clean_data_mean.csv
/kaggle/input/raw_data.csv
/kaggle/input/clean_data_GCN.csv
/kaggle/input/FillZero_minmax_baseline/clean_data_minmax_fill-zero.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/val/val_week1_2.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/test/test_week2.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/clean_data_week2.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_2.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_3.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_4.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_1.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_5.csv
/kaggle/input/FillZero_minmax_baseline/clean_week3/val/val_week1_2_3.csv
/kaggle/input/FillZero_minmax_baseline/clean_week3/test/test_week3.csv
/kaggle/input/FillZero_minmax_bas

In [2]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from kerastuner.tuners import RandomSearch
from sklearn.model_selection import StratifiedKFold
import time
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_recall_fscore_support, roc_auc_score

<ipython-input-2-63192117ceec>:7: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [3]:
# Biến global cho base path
BASE_PATH = "/kaggle/input/KNN_minmax_baseline"
# Tuần và số phần fold
weeks = ['week1', 'week2', 'week3', 'week4']
fold_parts = 5

# Tạo five_fold_files
five_fold_files = {
    week: [
        f"{BASE_PATH}/clean_{week}/train/5-folds/data_part_{i}.csv"
        for i in range(1, fold_parts + 1)
    ]
    for week in weeks
}

# Tạo file_validation
file_validation = {
    'week1': [f"{BASE_PATH}/clean_week1/val/val_week1.csv"],
    'week2': [f"{BASE_PATH}/clean_week2/val/val_week1_2.csv"],
    'week3': [f"{BASE_PATH}/clean_week3/val/val_week1_2_3.csv"],
    'week4': [f"{BASE_PATH}/clean_week4/val/val_week1_2_3_4.csv"]
}

# Tạo file_test
file_test = {
    week: [f"{BASE_PATH}/clean_{week}/test/test_{week}.csv"]
    for week in weeks
}

## Tìm siêu tham số tốt nhất cho từng tuần

In [4]:
# Định nghĩa Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred)
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
        return K.sum(loss, axis=-1)
    return focal_loss_fixed

# Tạo hàm train cho từng tuần
def train_week_model(week_number, file_paths_train, file_validataion):
    # Đọc dữ liệu
    train_data = pd.read_csv(file_paths_train)
    val_data = pd.read_csv(file_validataion)
    
    # Tách đặc trưng và nhãn
    X_train = train_data.drop(columns=["classification_encoded", "user_id", "course_id", "school", "enroll_time", "classification"])
    y_train = train_data["classification_encoded"]

    X_val = val_data.drop(columns=["classification_encoded", "user_id", "course_id", "school", "enroll_time", "classification"])
    y_val = val_data["classification_encoded"]
    
    # Áp dụng Over-sampling cho dữ liệu huấn luyện bằng SMOTE
    oversampler = SMOTE(sampling_strategy='auto', random_state=42)
    X_train_res, y_train_res = oversampler.fit_resample(X_train, y_train)
    
    # Reshape dữ liệu cho mô hình BiLSTM
    X_train_res = X_train_res.values.reshape(X_train_res.shape[0], X_train_res.shape[1], 1)
    X_val = X_val.values.reshape(X_val.shape[0], X_val.shape[1], 1)
    
    # One-hot encode nhãn
    y_train_res = tf.keras.utils.to_categorical(y_train_res, num_classes=5)
    y_val = tf.keras.utils.to_categorical(y_val, num_classes=5)
    
    def build_model(hp):
        inputs = tf.keras.Input(shape=(X_train_res.shape[1], 1))  # Khởi tạo đầu vào
        
        # RNN layer 1
        x = layers.SimpleRNN(
            units=hp.Int('units_1', min_value=32, max_value=256, step=32),
            return_sequences=True
        )(inputs)
        x = layers.Dropout(rate=hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1))(x)
        
        # RNN layer 2
        x = layers.SimpleRNN(
            units=hp.Int('units_2', min_value=32, max_value=256, step=32),
            return_sequences=False
        )(x)
        x = layers.Dropout(rate=hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1))(x)
        
        # Lớp đầu ra
        outputs = layers.Dense(5, activation='softmax')(x)
        
        # Khởi tạo mô hình
        model = tf.keras.Model(inputs=inputs, outputs=outputs)
        
        # Compile với Focal Loss
        model.compile(optimizer=tf.keras.optimizers.Adam(
                          learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
                      loss=focal_loss(gamma=2., alpha=0.25),
                      metrics=['accuracy'])
        
        return model

    
    # Khởi tạo RandomSearch tuner
    tuner = RandomSearch(
        build_model,
        objective='val_accuracy',
        max_trials=10,
        executions_per_trial=1,
        directory='my_dir',
        project_name=f'rnn_tuning_week{week_number}'
    )
    
    # Tìm kiếm siêu tham số tốt nhất
    tuner.search(X_train_res, y_train_res,
                 epochs=20,
                 validation_data=(X_val, y_val),
                 batch_size=32)
    
    # Trả về kết quả tối ưu cho tuần
    best_params = tuner.get_best_hyperparameters(num_trials=1)[0]
    return best_params

In [5]:
# Định nghĩa đường dẫn đến dữ liệu cho từng tuần
file_paths_train = {
    week: f"{BASE_PATH}/clean_{week}/train/clean_data_{week}.csv"
    for week in weeks
}

# Định nghĩa file_validation theo quy luật riêng
file_validation = {
    f"week{idx + 1}": f"{BASE_PATH}/clean_week{idx + 1}/val/val_week{'_'.join(str(i) for i in range(1, idx + 2))}.csv"
    for idx in range(len(weeks))
}

In [6]:
# Tìm tham số tốt nhất cho từng tuần
best_params_week1 = train_week_model(1, file_paths_train["week1"], file_validation["week1"])
best_params_week2 = train_week_model(2, file_paths_train["week2"], file_validation["week2"])
best_params_week3 = train_week_model(3, file_paths_train["week3"], file_validation["week3"])
best_params_week4 = train_week_model(4, file_paths_train["week4"], file_validation["week4"])

# In thông tin chi tiết các tham số tối ưu
print("Best Parameters for Week 1:")
for param_name in best_params_week1.values.keys():
    print(f"{param_name}: {best_params_week1.get(param_name)}")

print("\nBest Parameters for Week 2:")
for param_name in best_params_week2.values.keys():
    print(f"{param_name}: {best_params_week2.get(param_name)}")

print("\nBest Parameters for Week 3:")
for param_name in best_params_week3.values.keys():
    print(f"{param_name}: {best_params_week3.get(param_name)}")

print("\nBest Parameters for Week 4:")
for param_name in best_params_week4.values.keys():
    print(f"{param_name}: {best_params_week4.get(param_name)}")


Trial 10 Complete [00h 03m 20s]
val_accuracy: 0.9237340092658997

Best val_accuracy So Far: 0.9237340092658997
Total elapsed time: 00h 31m 37s
Best Parameters for Week 1:
units_1: 160
dropout_1: 0.30000000000000004
units_2: 128
dropout_2: 0.1
learning_rate: 0.0007129239549612552

Best Parameters for Week 2:
units_1: 96
dropout_1: 0.1
units_2: 224
dropout_2: 0.30000000000000004
learning_rate: 0.0005159173467296483

Best Parameters for Week 3:
units_1: 256
dropout_1: 0.4
units_2: 128
dropout_2: 0.30000000000000004
learning_rate: 0.0002444988826327385

Best Parameters for Week 4:
units_1: 224
dropout_1: 0.2
units_2: 192
dropout_2: 0.1
learning_rate: 0.00016164873790817211


## Danh sách tham số tốt nhất của từng tuần

In [7]:
# Danh sách tham số tốt nhất
best_params = {
    "week1": best_params_week1,
    "week2": best_params_week2,
    "week3": best_params_week3,
    "week4": best_params_week4
}

In [8]:
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras import backend as K

# Định nghĩa Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred)
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
        return K.sum(loss, axis=-1)
    
    return focal_loss_fixed

# Xây dựng mô hình BiLSTM
def build_RNN_model(params, input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    
    # RNN layer 1
    x = layers.SimpleRNN(
        units=params.get('units_1'),
        return_sequences=True
    )(inputs)
    x = layers.Dropout(rate=params.get('dropout_1', 0.2))(x)
    
    # RNN layer 2
    x = layers.SimpleRNN(
        units=params.get('units_2', 32),
        return_sequences=False
    )(x)
    x = layers.Dropout(rate=params.get('dropout_2', 0.2))(x)
    
    # Lớp đầu ra
    outputs = layers.Dense(5, activation='softmax')(x)
    
    # Khởi tạo mô hình
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    # Compile với Focal Loss
    model.compile(optimizer=tf.keras.optimizers.Adam(
                      learning_rate=params['learning_rate']),
                  loss=focal_loss(gamma=params.get('gamma', 2.), alpha=params.get('alpha', 0.25)),
                  metrics=['accuracy'])
    
    return model


In [9]:
# Biến lưu kết quả tổng quát
overall_results_5folds = []

# Lặp qua từng tuần
for week, file_paths in five_fold_files.items():
    print(f"\nProcessing {week} with best parameters...")
    params = best_params[week].values
    print(f"best parameters for {week}: {params}")
    
    # Biến lưu kết quả cho từng tuần
    week_results = {
        "week": week,
        "accuracy_per_fold": [],
        "precision_per_label": [],
        "recall_per_label": [],
        "f1_score_per_label": [],
        "auc_roc_per_label": [],    # AUC từng lớp
        "auc_roc_macro": [],        # AUC macro
        "auc_roc_weighted": [],     # AUC weighted (tự tính)
        "precision_macro": [],
        "recall_macro": [],
        "f1_macro": [],
        "precision_weighted": [],
        "recall_weighted": [],
        "f1_weighted": [],
        "confusion_matrices": [],
        "train_times": [],
        "test_times": []
    }

    # Lặp qua từng fold
    for i in range(len(file_paths)):
        print(f"Fold {i+1}: Using file {file_paths[i]} as test set")
        
        # Tải dữ liệu
        test_data = pd.read_csv(file_paths[i])
        train_data = pd.concat([pd.read_csv(file_paths[j]) for j in range(len(file_paths)) if j != i])
        
        # Tách X và y
        X_train = train_data.drop(columns=["classification_encoded", "user_id",
                                           "course_id", "school", "enroll_time", "classification"])
        y_train = to_categorical(train_data['classification_encoded'], num_classes=5)
        
        X_test = test_data.drop(columns=["classification_encoded", "user_id",
                                         "course_id", "school", "enroll_time", "classification"])
        y_test = to_categorical(test_data['classification_encoded'], num_classes=5)

        # Reshape dữ liệu cho LSTM
        X_train = X_train.to_numpy().reshape((X_train.shape[0], 1, X_train.shape[1]))
        X_test = X_test.to_numpy().reshape((X_test.shape[0], 1, X_test.shape[1]))
        input_shape = (X_train.shape[1], X_train.shape[2])  # Lấy kích thước từ dữ liệu thực tế
        # Xây dựng mô hình với tham số tốt nhất
        model = build_RNN_model(params, input_shape)
        
        # Bắt đầu tính thời gian huấn luyện
        start_train = time.time()
        model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), batch_size=32)
        end_train = time.time()
        
        # Bắt đầu tính thời gian kiểm thử
        start_test = time.time()
        y_pred = model.predict(X_test)
        end_test = time.time()
        
        # Tính thời gian và lưu lại
        train_time = end_train - start_train
        test_time = end_test - start_test
        week_results["train_times"].append(train_time)
        week_results["test_times"].append(test_time)

        # Đánh giá mô hình trên tập kiểm thử của fold hiện tại
        _, accuracy = model.evaluate(X_test, y_test, verbose=0)
        week_results["accuracy_per_fold"].append(accuracy)
        
        # Dự đoán
        y_pred_classes = y_pred.argmax(axis=1)
        y_test_classes = y_test.argmax(axis=1)
        
        # Tính các chỉ số cho mỗi fold
        precision, recall, f1, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average=None)
        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='macro')
        precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='weighted')
        conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
        
        # Tính AUC-ROC
        try:
            # Tính AUC macro và theo từng lớp với OvR
            auc_macro = roc_auc_score(y_test, y_pred, multi_class="ovr", average="macro")
            auc_per_class = roc_auc_score(y_test, y_pred, multi_class="ovr", average=None)
            # Tính AUC weighted: tính trọng số theo số mẫu của từng lớp
            supports = np.bincount(y_test_classes, minlength=5)
            auc_weighted = np.sum(auc_per_class * supports) / np.sum(supports)
        except Exception as e:
            print(f"Lỗi khi tính AUC: {e}")
            auc_macro = np.nan
            auc_per_class = [np.nan] * 5
            auc_weighted = np.nan
            
        # Lưu kết quả của fold hiện tại
        week_results["precision_per_label"].append(precision)
        week_results["recall_per_label"].append(recall)
        week_results["f1_score_per_label"].append(f1)
        week_results["auc_roc_per_label"].append(auc_per_class)  # AUC từng lớp
        week_results["auc_roc_macro"].append(auc_macro)          # AUC macro
        week_results["auc_roc_weighted"].append(auc_weighted)      # AUC weighted
        week_results["confusion_matrices"].append(conf_matrix)
        week_results["precision_macro"].append(precision_macro)
        week_results["recall_macro"].append(recall_macro)
        week_results["f1_macro"].append(f1_macro)
        week_results["precision_weighted"].append(precision_weighted)
        week_results["recall_weighted"].append(recall_weighted)
        week_results["f1_weighted"].append(f1_weighted)

    # Tính trung bình cho từng nhãn
    average_precision_per_label = np.mean(week_results["precision_per_label"], axis=0)
    average_recall_per_label = np.nanmean(week_results["recall_per_label"], axis=0)
    average_f1_per_label = np.nanmean(week_results["f1_score_per_label"], axis=0)
    average_auc_per_label = np.nanmean(week_results["auc_roc_per_label"], axis=0)
    average_confusion_matrix = np.nanmean(week_results["confusion_matrices"], axis=0)
    average_train_time = sum(week_results["train_times"]) / len(week_results["train_times"])
    average_test_time = sum(week_results["test_times"]) / len(week_results["test_times"])
    average_accuracy = np.nanmean(week_results["accuracy_per_fold"])
    average_precision_macro = np.nanmean(week_results["precision_macro"])
    average_recall_macro = np.nanmean(week_results["recall_macro"])
    average_f1_macro = np.nanmean(week_results["f1_macro"])
    average_auc_macro = np.nanmean(week_results["auc_roc_macro"])
    average_precision_weighted = np.nanmean(week_results["precision_weighted"])
    average_recall_weighted = np.nanmean(week_results["recall_weighted"])
    average_f1_weighted = np.nanmean(week_results["f1_weighted"])
    average_auc_weighted = np.nanmean(week_results["auc_roc_weighted"])


    # Tạo DataFrame cho precision, recall, f1-score
    labels = np.unique(y_test_classes)  # Lấy nhãn từ y_test_classes
    metrics_df = pd.DataFrame({
        "Label": labels,
        "Average Precision": average_precision_per_label,
        "Average Recall": average_recall_per_label,
        "Average F1-Score": average_f1_per_label,
        "Average AUC": average_auc_per_label
    })
    
    # Tạo DataFrame cho confusion matrix
    confusion_df = pd.DataFrame(average_confusion_matrix, index=labels, columns=labels)
    # In kết quả Accuracy và Macro metrics
    print("\n=== Average Accuracy ===")
    print(f"{average_accuracy:.4f}")
    print("\n=== Average Macro Metrics ===")
    print(f"Macro Precision: {average_precision_macro:.4f}")
    print(f"Macro Recall: {average_recall_macro:.4f}")
    print(f"Macro F1-Score: {average_f1_macro:.4f}")
    print(f"Macro AUC-ROC: {average_auc_macro:.4f}")
    print("\n=== Average Weighted Metrics ===")
    print(f"Weighted Precision: {average_precision_weighted:.4f}")
    print(f"Weighted Recall: {average_recall_weighted:.4f}")
    print(f"Weighted F1-Score: {average_f1_weighted:.4f}")
    print(f"Weighted AUC-ROC: {average_auc_weighted:.4f}")
    print("\n=== Average Metrics per Label ===")
    print(metrics_df)
    print("\n=== Average Confusion Matrix ===")
    print(confusion_df)
    
    # Cập nhật kết quả cho tuần hiện tại
    week_results.update({
        "average_accuracy": average_accuracy,
        "average_precision_macro": average_precision_macro,
        "average_recall_macro": average_recall_macro,
        "average_f1_macro": average_f1_macro,
        "average_auc_macro": average_auc_macro,
        "average_precision_weighted": average_precision_weighted,
        "average_recall_weighted": average_recall_weighted,
        "average_f1_weighted": average_f1_weighted,
        "average_auc_weighted": average_auc_weighted,
        "average_metrics_df": metrics_df,
        "average_confusion_matrix": confusion_df,
        "average_train_times": average_train_time,
        "average_test_times": average_test_time,
    })
    overall_results_5folds.append(week_results)


Processing week1 with best parameters...
best parameters for week1: {'units_1': 160, 'dropout_1': 0.30000000000000004, 'units_2': 128, 'dropout_2': 0.1, 'learning_rate': 0.0007129239549612552}
Fold 1: Using file /kaggle/input/KNN_minmax_baseline/clean_week1/train/5-folds/data_part_1.csv as test set
Epoch 1/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 15s 32ms/step - accuracy: 0.5745 - loss: 0.1746 - val_accuracy: 0.6653 - val_loss: 0.1233
Epoch 2/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6347 - loss: 0.1369 - val_accuracy: 0.6733 - val_loss: 0.1175
Epoch 3/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6450 - loss: 0.1313 - val_accuracy: 0.6740 - val_loss: 0.1248
Epoch 4/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6530 - loss: 0.1291 - val_accuracy: 0.6740 - val_loss: 0.1150
Epoch 5/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6542 - loss: 0.1291 - val_accuracy: 0.6916 - val_loss: 0.1136
Epoch 6/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


328/328 ━━━━━━━━━━━━━━━━━━━━ 13s 29ms/step - accuracy: 0.5691 - loss: 0.1763 - val_accuracy: 0.6809 - val_loss: 0.1233
Epoch 2/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6336 - loss: 0.1361 - val_accuracy: 0.6733 - val_loss: 0.1208
Epoch 3/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6410 - loss: 0.1319 - val_accuracy: 0.6836 - val_loss: 0.1191
Epoch 4/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6596 - loss: 0.1240 - val_accuracy: 0.6874 - val_loss: 0.1161
Epoch 5/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6657 - loss: 0.1220 - val_accuracy: 0.6958 - val_loss: 0.1131
Epoch 6/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6756 - loss: 0.1212 - val_accuracy: 0.7011 - val_loss: 0.1118
Epoch 7/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6901 - loss: 0.1131 - val_accuracy: 0.7289 - val_loss: 0.1033
Epoch 8/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6935 - loss: 0.1133 - val_accuracy: 0.7457 - va

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


328/328 ━━━━━━━━━━━━━━━━━━━━ 13s 30ms/step - accuracy: 0.5657 - loss: 0.1794 - val_accuracy: 0.6861 - val_loss: 0.1266
Epoch 2/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6436 - loss: 0.1328 - val_accuracy: 0.6777 - val_loss: 0.1175
Epoch 3/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6535 - loss: 0.1271 - val_accuracy: 0.6983 - val_loss: 0.1194
Epoch 4/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6595 - loss: 0.1249 - val_accuracy: 0.6945 - val_loss: 0.1133
Epoch 5/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6818 - loss: 0.1181 - val_accuracy: 0.7121 - val_loss: 0.1054
Epoch 6/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6985 - loss: 0.1172 - val_accuracy: 0.7166 - val_loss: 0.1063
Epoch 7/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6898 - loss: 0.1190 - val_accuracy: 0.7365 - val_loss: 0.1015
Epoch 8/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7188 - loss: 0.1089 - val_accuracy: 0.6979 - va

...

Fold 2: Using file /kaggle/input/KNN_minmax_baseline/clean_week4/train/5-folds/data_part_2.csv as test set
Epoch 1/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 13s 28ms/step - accuracy: 0.5853 - loss: 0.1603 - val_accuracy: 0.6862 - val_loss: 0.1221
Epoch 2/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6570 - loss: 0.1315 - val_accuracy: 0.7286 - val_loss: 0.1120
Epoch 3/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6815 - loss: 0.1229 - val_accuracy: 0.7564 - val_loss: 0.0983
Epoch 4/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7119 - loss: 0.1141 - val_accuracy: 0.8048 - val_loss: 0.0897
Epoch 5/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7431 - loss: 0.1047 - val_accuracy: 0.8044 - val_loss: 0.0854
Epoch 6/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7563 - loss: 0.0977 - val_accuracy: 0.8025 - val_loss: 0.0896
Epoch 7/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7659 - loss: 0.0921 - val_accuracy: 0.8025 - val_loss: 0.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fold 3: Using file /kaggle/input/KNN_minmax_baseline/clean_week4/train/5-folds/data_part_3.csv as test set
Epoch 1/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 15s 33ms/step - accuracy: 0.5771 - loss: 0.1616 - val_accuracy: 0.6690 - val_loss: 0.1205
Epoch 2/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6619 - loss: 0.1274 - val_accuracy: 0.6957 - val_loss: 0.1151
Epoch 3/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6673 - loss: 0.1249 - val_accuracy: 0.7189 - val_loss: 0.1080
Epoch 4/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7181 - loss: 0.1105 - val_accuracy: 0.7777 - val_loss: 0.0934
Epoch 5/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7481 - loss: 0.1019 - val_accuracy: 0.7876 - val_loss: 0.0896
Epoch 6/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7530 - loss: 0.1010 - val_accuracy: 0.8055 - val_loss: 0.0896
Epoch 7/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7668 - loss: 0.0942 - val_accuracy: 0.8028 - val_loss: 0.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 13s 30ms/step - accuracy: 0.6024 - loss: 0.1527 - val_accuracy: 0.7014 - val_loss: 0.1132
Epoch 2/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6612 - loss: 0.1295 - val_accuracy: 0.7059 - val_loss: 0.1092
Epoch 3/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6907 - loss: 0.1184 - val_accuracy: 0.7540 - val_loss: 0.0959
Epoch 4/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7299 - loss: 0.1072 - val_accuracy: 0.7925 - val_loss: 0.0888
Epoch 5/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7586 - loss: 0.1009 - val_accuracy: 0.7902 - val_loss: 0.0847
Epoch 6/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7685 - loss: 0.0965 - val_accuracy: 0.8150 - val_loss: 0.0819
Epoch 7/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7755 - loss: 0.0926 - val_accuracy: 0.8105 - val_loss: 0.0812
Epoch 8/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7774 - loss: 0.0895 - val_accuracy: 

## Kết quả cross validation trên 5-folds

In [10]:
# Duyệt qua các tuần trong overall_results
for week_result in overall_results_5folds:
    week = week_result["week"]
    average_train_time = np.mean(week_result["train_times"])
    average_test_time = np.mean(week_result["test_times"])
    average_metrics_df = week_result["average_metrics_df"]
    average_accuracy = np.mean(week_results["accuracy_per_fold"])
    average_confusion_matrix = week_result["average_confusion_matrix"]
    
    # In kết quả
    print(f"\n=== Results for {week} ===")
    print(f"Average Accurancy: {average_accuracy}")
    print(f"Average Train Time: {average_train_time:.4f} seconds")
    print(f"Average Test Time: {average_test_time:.4f} seconds")
    print(f"Average AUC Macro: {average_auc_macro}")
    print(f"Average AUC Weighted: {average_auc_weighted}")
    print("\nAverage Precision, Recall, F1-Score, AUC-ROC per Label:")
    print(average_metrics_df)
    print("\nAverage Confusion Matrix:")
    print(average_confusion_matrix)



=== Results for week1 ===
Average Accurancy: 0.8457905769348144
Average Train Time: 25.7571 seconds
Average Test Time: 0.7962 seconds
Average AUC Macro: 0.9236904968439829
Average AUC Weighted: 0.9501200481990416

Average Precision, Recall, F1-Score, AUC-ROC per Label:
   Label  Average Precision  Average Recall  Average F1-Score  Average AUC
0      0           0.677147        0.607000          0.622204     0.874828
1      1           0.000000        0.000000          0.000000     0.834351
2      2           0.657641        0.413695          0.500876     0.856738
3      3           0.545909        0.464671          0.482372     0.872062
4      4           0.820141        0.919420          0.864818     0.911405

Average Confusion Matrix:
       0    1     2     3       4
0  364.2  0.4  14.8  29.2   191.4
1   34.0  0.0   1.4   9.6    42.6
2   38.2  0.0  68.0  10.2    48.0
3   35.6  0.0   2.4  77.6    51.4
4   84.0  0.0  17.6  27.6  1474.2

=== Results for week2 ===
Average Accurancy: 0.

## Kiểm tra trên tập test

In [11]:
# Mảng lưu dữ liệu của các tuần
results = []

def process_week(week_num, best_params, results):
    print(f"\n=== Processing Week {week_num} ===")
    params = best_params[f"week{week_num}"].values
    # Đường dẫn tới dữ liệu tuần tương ứng
    train_path = f"{BASE_PATH}/clean_week{week_num}/train/clean_data_week{week_num}.csv"
    test_path = f"{BASE_PATH}/clean_week{week_num}/test/test_week{week_num}.csv"
    
    # Load dữ liệu
    train_data = pd.read_csv(train_path)
    test_data = pd.read_csv(test_path)
    
    # Tách X và y
    X_train = train_data.drop(columns=["classification_encoded", "user_id",
                                       "course_id", "school", "enroll_time", "classification"])
    y_train = train_data['classification_encoded']
    
    X_test = test_data.drop(columns=["classification_encoded", "user_id",
                                     "course_id", "school", "enroll_time", "classification"])
    y_test = test_data['classification_encoded']

    # Áp dụng SMOTE cho tập huấn luyện
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

    # Chuyển đổi nhãn sang dạng one-hot
    y_train_resampled = to_categorical(y_train_resampled, num_classes=5)
    y_test = to_categorical(y_test, num_classes=5)

    # Reshape dữ liệu cho LSTM
    X_train_resampled = X_train_resampled.to_numpy().reshape((X_train_resampled.shape[0], 1, X_train_resampled.shape[1]))
    X_test = X_test.to_numpy().reshape((X_test.shape[0], 1, X_test.shape[1]))
    
    # Xây dựng mô hình với tham số tốt nhất
    input_shape = (X_train_resampled.shape[1], X_train_resampled.shape[2])  # Lấy kích thước từ dữ liệu thực tế
    model = build_RNN_model(params, input_shape)
    
    # Huấn luyện mô hình
    start_train = time.time()
    model.fit(X_train_resampled, y_train_resampled, epochs=20, validation_split=0.1, batch_size=32)
    end_train = time.time()
    
    # Kiểm thử mô hình
    start_test = time.time()
    y_pred = model.predict(X_test)
    end_test = time.time()
    
    # Tính thời gian huấn luyện và kiểm thử
    train_time = end_train - start_train
    test_time = end_test - start_test
    
    # Đánh giá mô hình
    y_pred_classes = y_pred.argmax(axis=1)
    y_test_classes = y_test.argmax(axis=1)
    
    precision, recall, f1, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average=None)
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='macro')
    precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='weighted')
    conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
    accuracy = accuracy_score(y_test_classes, y_pred_classes)
    
    # Tính AUC-ROC (với one-vs-rest)
    try:
        auc_macro = roc_auc_score(y_test, y_pred, multi_class="ovr", average="macro")
        auc_per_class = roc_auc_score(y_test, y_pred, multi_class="ovr", average=None)
        # Tính AUC weighted tự tính theo trọng số mẫu của từng lớp
        supports = np.bincount(y_test_classes, minlength=5)
        auc_weighted = np.sum(auc_per_class * supports) / np.sum(supports)
    except Exception as e:
        print(f"Lỗi khi tính AUC: {e}")
        auc_macro = np.nan
        auc_per_class = [np.nan] * 5
        auc_weighted = np.nan

    # Lưu kết quả vào mảng
    results.append({
        "week": week_num,
        "train_time": train_time,
        "test_time": test_time,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "accuracy": accuracy,
        "precision_macro": precision_macro,
        "recall_macro": recall_macro,
        "f1_macro": f1_macro,
        "precision_weighted": precision_weighted,
        "recall_weighted": recall_weighted,
        "f1_weighted": f1_weighted,
        "auc_macro": auc_macro,
        "auc_weighted": auc_weighted,
        "auc_per_class": auc_per_class,
        "confusion_matrix": conf_matrix
    })
    
    # In kết quả chi tiết
    print("\n=== Precision, Recall, F1-Score per Label ===")
    print(pd.DataFrame({
        "Label": np.unique(y_test_classes),
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1
    }))

    print("\n=== Macro Averages & Accuracy ===")
    print(f"Macro Precision: {precision_macro:.4f}")
    print(f"Macro Recall: {recall_macro:.4f}")
    print(f"Macro F1-Score: {f1_macro:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    
    print("\n=== Weighted Averages ===")
    print(f"Weighted Precision: {precision_weighted:.4f}")
    print(f"Weighted Recall: {recall_weighted:.4f}")
    print(f"Weighted F1-Score: {f1_weighted:.4f}")
    
    print("\n=== AUC-ROC ===")
    print(f"AUC Macro: {auc_macro:.4f}")
    print(f"AUC Weighted: {auc_weighted:.4f}")
    print(f"AUC per Label: {auc_per_class}")
    
    print("\n=== Confusion Matrix ===")
    print(pd.DataFrame(conf_matrix, index=np.unique(y_test_classes), columns=np.unique(y_test_classes)))
    
    print(f"\nTrain Time: {train_time:.2f} seconds")
    print(f"Test Time: {test_time:.2f} seconds")

In [12]:
process_week(1, best_params, results)


=== Processing Week 1 ===
Epoch 1/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.4066 - loss: 0.2295 - val_accuracy: 0.4445 - val_loss: 0.3212
Epoch 2/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.4864 - loss: 0.1962 - val_accuracy: 0.5198 - val_loss: 0.2136
Epoch 3/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5345 - loss: 0.1770 - val_accuracy: 0.4712 - val_loss: 0.2793
Epoch 4/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5679 - loss: 0.1657 - val_accuracy: 0.5702 - val_loss: 0.2332
Epoch 5/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5814 - loss: 0.1603 - val_accuracy: 0.5253 - val_loss: 0.2588
Epoch 6/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5948 - loss: 0.1564 - val_accuracy: 0.4295 - val_loss: 0.3160
Epoch 7/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6056 - loss: 0.1530 - val_accuracy: 0.5413 - val_loss: 0.2570
Epoch 8/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accura

In [13]:
process_week(2, best_params, results)


=== Processing Week 2 ===
Epoch 1/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.4198 - loss: 0.2247 - val_accuracy: 0.4443 - val_loss: 0.2657
Epoch 2/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5428 - loss: 0.1733 - val_accuracy: 0.5121 - val_loss: 0.2338
Epoch 3/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5668 - loss: 0.1610 - val_accuracy: 0.5261 - val_loss: 0.2727
Epoch 4/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5873 - loss: 0.1547 - val_accuracy: 0.6560 - val_loss: 0.2347
Epoch 5/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6065 - loss: 0.1463 - val_accuracy: 0.5395 - val_loss: 0.2711
Epoch 6/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6211 - loss: 0.1408 - val_accuracy: 0.6194 - val_loss: 0.2078
Epoch 7/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6320 - loss: 0.1358 - val_accuracy: 0.5061 - val_loss: 0.2565
Epoch 8/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accura

In [14]:
process_week(3, best_params, results)


=== Processing Week 3 ===
Epoch 1/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.3576 - loss: 0.2792 - val_accuracy: 0.4225 - val_loss: 0.3248
Epoch 2/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.4704 - loss: 0.2081 - val_accuracy: 0.4350 - val_loss: 0.3346
Epoch 3/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5262 - loss: 0.1806 - val_accuracy: 0.4428 - val_loss: 0.3311
Epoch 4/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5640 - loss: 0.1646 - val_accuracy: 0.5620 - val_loss: 0.2733
Epoch 5/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5811 - loss: 0.1540 - val_accuracy: 0.6141 - val_loss: 0.2087
Epoch 6/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5954 - loss: 0.1515 - val_accuracy: 0.6094 - val_loss: 0.2248
Epoch 7/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6100 - loss: 0.1438 - val_accuracy: 0.6618 - val_loss: 0.1997
Epoch 8/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accura

In [15]:
process_week(4, best_params, results)


=== Processing Week 4 ===
Epoch 1/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.4209 - loss: 0.2273 - val_accuracy: 0.4388 - val_loss: 0.2693
Epoch 2/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5462 - loss: 0.1763 - val_accuracy: 0.5752 - val_loss: 0.2392
Epoch 3/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5886 - loss: 0.1528 - val_accuracy: 0.5178 - val_loss: 0.2467
Epoch 4/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6119 - loss: 0.1410 - val_accuracy: 0.5722 - val_loss: 0.1994
Epoch 5/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6256 - loss: 0.1362 - val_accuracy: 0.6219 - val_loss: 0.2182
Epoch 6/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6537 - loss: 0.1266 - val_accuracy: 0.4996 - val_loss: 0.2711
Epoch 7/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6647 - loss: 0.1228 - val_accuracy: 0.5894 - val_loss: 0.2176
Epoch 8/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accura

In [16]:
# Hiển thị dữ liệu của các tuần
print("\n=== Summary Results for All Weeks ===")
for result in results:
    print(f"Week {result['week']}:")
    print(f"  Train Time: {result['train_time']:.2f} seconds")
    print(f"  Test Time: {result['test_time']:.2f} seconds")
    print(f"  Accurancy: {result['accuracy']}")
    print(f"  Precision: {result['precision']}")
    print(f"  Recall: {result['recall']}")
    print(f"  F1-Score: {result['f1_score']}")
    print(f"  Macro Precision: {result['precision_macro']}")
    print(f"  Macro Recall: {result['recall_macro']}")
    print(f"  Macro F1-Score: {result['f1_macro']}")
    print(f"  Confusion Matrix:\n{result['confusion_matrix']}")
    print("\n=== AUC-ROC ===")
    print(f"AUC Macro: {result['auc_macro']:.4f}")
    print(f"AUC Weighted: {result['auc_weighted']:.4f}")
    print(f"AUC per Label: {result['auc_per_class']}")


=== Summary Results for All Weeks ===
Week 1:
  Train Time: 45.89 seconds
  Test Time: 0.99 seconds
  Accurancy: 0.7597560975609756
  Precision: [0.80756014 0.25146199 0.51724138 0.51111111 0.90507012]
  Recall: [0.62666667 0.7962963  0.58252427 0.65714286 0.83649053]
  F1-Score: [0.70570571 0.38222222 0.54794521 0.575      0.86943005]
  Macro Precision: 0.5984889469689892
  Macro Recall: 0.6998241240730472
  Macro F1-Score: 0.6160606370441702
  Confusion Matrix:
[[235  64  19   4  53]
 [  3  43   1   1   6]
 [ 11  16  60   3  13]
 [  8   4   8  69  16]
 [ 34  44  28  58 839]]

=== AUC-ROC ===
AUC Macro: 0.9127
AUC Weighted: 0.9262
AUC per Label: [0.93143821 0.90837654 0.88823266 0.90381883 0.93146933]
Week 2:
  Train Time: 46.11 seconds
  Test Time: 1.00 seconds
  Accurancy: 0.7567073170731707
  Precision: [0.83524904 0.2        0.42405063 0.58267717 0.92165397]
  Recall: [0.58133333 0.64814815 0.65048544 0.7047619  0.8444666 ]
  F1-Score: [0.68553459 0.30567686 0.51340996 0.63793103